In [ ]:
from transformers import GemmaForCausalLM,GemmaTokenizer,BitsAndBytesConfig
import torch
from peft import LoraConfig,get_peft_model,prepare_model_for_kbit_training

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
)
tokenizer = GemmaTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = GemmaForCausalLM.from_pretrained("google/gemma-1.1-2b-it",quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model)

In [ ]:
special_tokens_dict = ["<STT>"]
num_added_toks = tokenizer.add_tokens(special_tokens_dict,special_tokens=True)
print("We have added", num_added_toks, "tokens")
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer.special_tokens_map

In [ ]:
prompt = "I know <STT> is a good person."
input_ids = tokenizer.encode(prompt, return_tensors="pt",skip_special_tokens=True)
print(tokenizer.decode(input_ids[0],skip_special_tokens=True))

: 

In [ ]:
prompt = ["I like deep learning", "I like to play with my dog", "I like to play with my cat", "I like to play with my"]
tokenizer.padding_side = "left"
tokens = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
mask=tokens.attention_mask

In [ ]:
print(mask)
mask.sum(-1)

In [ ]:
input = tokenizer("You can generate the output not include the input?",add_special_tokens=True,return_tensors='pt')

In [ ]:
tokenizer.batch_decode(input["input_ids"],skip_special_tokens=False)

In [ ]:
fwd = model(**input,return_dict=True)

In [ ]:
fwd.logits
torch.argmax(torch.softmax(fwd.logits,dim=-1),dim=-1)

In [ ]:
tokenizer.batch_decode(torch.argmax(torch.softmax(fwd.logits,dim=-1),dim=-1),skip_special_tokens=False)

In [ ]:
gt = model.generate(input["input_ids"],do_sample=True,max_length=100,num_return_sequences=1,top_k=50,top_p=0.95,temperature=0.9)

In [ ]:
tokenizer.decode(gt[0],skip_special_tokens=False)

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
a = torch.tensor([4,5,6])
b = torch.tensor([1,2])
pad_sequence([a,b],batch_first=True,padding_value=0)

tensor([[4, 5, 6],
        [1, 2, 0]])

In [6]:
a = torch.tensor([[4,5,6],[1,2,0]])
print(a.shape)

torch.Size([2, 3])
